In [ ]:
#IMPORTING REQUIRED LIBRARIES
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
import math

from lightgbm.sklearn import LGBMRegressor
from lightgbm.sklearn import LGBMClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from sklearn.metrics import roc_auc_score,accuracy_score

from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

import gc
gc.enable()


import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
#All functions

#FUNCTION FOR PROVIDING FEATURE SUMMARY
def feature_summary(df_fa):
    print('DataFrame shape')
    print('rows:',df_fa.shape[0])
    print('cols:',df_fa.shape[1])
    col_list=['Null','Unique_Count','Data_type','Max/Min','Mean','Std','Skewness','Sample_values']
    df=pd.DataFrame(index=df_fa.columns,columns=col_list)
    df['Null']=list([len(df_fa[col][df_fa[col].isnull()]) for i,col in enumerate(df_fa.columns)])
    #df['%_Null']=list([len(df_fa[col][df_fa[col].isnull()])/df_fa.shape[0]*100 for i,col in enumerate(df_fa.columns)])
    df['Unique_Count']=list([len(df_fa[col].unique()) for i,col in enumerate(df_fa.columns)])
    df['Data_type']=list([df_fa[col].dtype for i,col in enumerate(df_fa.columns)])
    for i,col in enumerate(df_fa.columns):
        if 'float' in str(df_fa[col].dtype) or 'int' in str(df_fa[col].dtype):
            df.at[col,'Max/Min']=str(round(df_fa[col].max(),2))+'/'+str(round(df_fa[col].min(),2))
            df.at[col,'Mean']=df_fa[col].mean()
            df.at[col,'Std']=df_fa[col].std()
            df.at[col,'Skewness']=df_fa[col].skew()
        df.at[col,'Sample_values']=list(df_fa[col].unique())
    display(df_fa.head())   
    return(df.fillna('-'))

In [ ]:
#DATASET VIEW
path1="../input/"
data_files=list(os.listdir(path1))
df_files=pd.DataFrame(data_files,columns=['File_Name'])
df_files['Size_in_MB']=df_files.File_Name.apply(lambda x:round(os.stat(path1+x).st_size/(1024*1024),2))
df_files

In [ ]:
# del df_train,df_test,df_submission,fs_train
# gc.collect()

In [ ]:
%%time
#READING AVAILABLE FILES DATASET
print('reading train dataset...')
df_train=pd.read_csv(path1+'train.csv',nrows=10000)
# print('reading test dataset...')
# df_test=pd.read_csv(path1+'test.csv',nrows=10000)
# print('submission file')
# df_submission=pd.read_csv(path1+'sample_submission.csv')

In [ ]:
fs_train=feature_summary(df_train)

In [ ]:
cat_features=list(fs_train[['Null','Data_type','Unique_Count','Sample_values']][fs_train.Data_type=='object'].index)

In [ ]:
noncat_features=list(fs_train[['Null','Data_type','Unique_Count','Sample_values']][fs_train.Data_type!='object'].index)

In [ ]:
cat_features

In [ ]:
noncat_features

In [ ]:
del df_train
gc.collect()

In [ ]:
%%time
#READING AVAILABLE FILES DATASET
print('reading train dataset...')
df_train=pd.read_csv(path1+'train.csv',usecols=['Platform','SkuEdition','HasDetections'])
# print('reading test dataset...')
# df_test=pd.read_csv(path1+'test.csv',nrows=10000)
# print('submission file')
# df_submission=pd.read_csv(path1+'sample_submission.csv')

In [ ]:
feature_summary(df_train)

In [ ]:
engineversion_unique=list(df_train['SkuEdition'].unique())

In [ ]:
for en_ver in engineversion_unique:
    print(en_ver)
    display(feature_summary(df_train[(df_train.Platform=='windows10') & (df_train.SkuEdition==en_ver)]))